
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-26 11:56:59.866320: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:56:59.899700: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 11:56:59.899753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 11:56:59.900990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 11:56:59.907019: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 11:56:59.907260: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=-0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  0.5931080199989083
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.028740191000906634
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.029045220999250887
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.02687659799994435
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.027408699999796227
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.02665285200055223
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_0_0,-0.111669,-1.168487,0.010955,-0.404452,1.230290,1.338637,0.505438,0.946543
model_2_0_1,-0.030727,-0.889549,-0.020687,-0.280481,1.140711,1.166444,0.521607,0.862992
model_2_0_2,0.027893,-0.485041,-0.298932,-0.183614,1.075835,0.916736,0.663801,0.797707
model_2_0_3,0.034908,-0.499635,-0.271421,-0.180875,1.068073,0.925745,0.649742,0.795861
model_2_0_4,0.050488,-0.478194,-0.247219,-0.161842,1.050830,0.912510,0.637374,0.783034
model_2_0_5,0.079529,-0.467379,-0.179063,-0.132277,1.018690,0.905833,0.602544,0.763109
model_2_0_6,0.091283,-0.574360,-0.129238,-0.166375,1.005681,0.971874,0.577081,0.786089
model_2_0_7,0.103552,-0.541958,-0.099124,-0.139917,0.992104,0.951871,0.561692,0.768258
model_2_0_8,0.104842,-0.547543,-0.091117,-0.139768,0.990676,0.955319,0.557600,0.768157
model_2_0_9,0.105150,-0.548298,-0.088998,-0.139379,0.990335,0.955786,0.556517,0.767895


DataFrame salvo em ../results/metrics_2_0
+++++++++++ [2_1] | 1 ++++++++++++++++++
Elapsed time:  0.44657151799947314
Stopped at epoch:  14
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_1] | 2 ++++++++++++++++++
Elapsed time:  0.22332784299942432
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_1] | 3 ++++++++++++++++++
Elapsed time:  0.2308503500007646
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_1] | 4 ++++++++++++++++++
Elapsed time:  0.26703320600063307
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 5 ++++++++++++++++++
Elapsed time:  0.4103622870006802
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [2_1] | 6 ++++++++++++++++++
Elapsed time:  0.3433486680005444
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_1] | 7 ++++++++++++++++++
Elapsed time:  0.24139974

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_1_24,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_22,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_21,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_20,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_19,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_18,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_17,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_16,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_23,-0.001237,-0.747432,-0.334943,-0.430438,1.108074,1.724910,0.292100,1.050646
model_2_1_15,0.000508,-0.735028,-0.336046,-0.421769,1.106143,1.712665,0.292342,1.044279


DataFrame salvo em ../results/metrics_2_1
+++++++++++ [2_2] | 1 ++++++++++++++++++
Elapsed time:  0.03630033599984017
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_2] | 2 ++++++++++++++++++
Elapsed time:  0.030495422999592847
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_2] | 3 ++++++++++++++++++
Elapsed time:  0.028203138001117622
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_2] | 4 ++++++++++++++++++
Elapsed time:  0.02819250100037607
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_2] | 5 ++++++++++++++++++
Elapsed time:  0.028972543999771005
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_2] | 6 ++++++++++++++++++
Elapsed time:  0.028778920999684487
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_2] | 7 ++++++++++++++++++
Elapsed time:  0.02

...

DataFrame salvo em ../results/metrics_12_7
+++++++++++ [12_8] | 1 ++++++++++++++++++
Elapsed time:  0.04062715100008063
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_8] | 2 ++++++++++++++++++
Elapsed time:  0.03281310900092649
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_8] | 3 ++++++++++++++++++
Elapsed time:  0.03191121899908467
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_8] | 4 ++++++++++++++++++
Elapsed time:  0.03241405900007521
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_8] | 5 ++++++++++++++++++
Elapsed time:  0.031126167999900645
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_8] | 6 ++++++++++++++++++
Elapsed time:  0.0317206729996542
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [12_8] | 7 ++++++++++++++++++
Elapsed time:  

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_8_0,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_22,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_21,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_20,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_19,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_18,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_17,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_16,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_15,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453
model_12_8_14,0.353672,-1.292085,0.588789,0.429284,0.715295,0.307139,0.664805,0.475453


DataFrame salvo em ../results/metrics_12_8
+++++++++++ [12_9] | 1 ++++++++++++++++++
Elapsed time:  0.0395069969999895
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 2 ++++++++++++++++++
Elapsed time:  0.030848922999211936
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 3 ++++++++++++++++++
Elapsed time:  0.03000843199879455
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_9] | 4 ++++++++++++++++++
Elapsed time:  0.031168990999503876
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 5 ++++++++++++++++++
Elapsed time:  0.03075944400006847
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_9] | 6 ++++++++++++++++++
Elapsed time:  0.037654278999980306
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 7 ++++++++++++++++++
Elapsed time:

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_9_0,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_22,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_21,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_20,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_19,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_18,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_17,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_16,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_15,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796
model_12_9_14,0.353672,0.246526,0.341828,0.311279,0.715295,0.707713,0.92074,0.80796


DataFrame salvo em ../results/metrics_12_9
